In [1]:
import torch
from torch.nn import functional as F
from torch import nn

d_model = 512
block_size = 256
batch_size = 32 
n_heads = 8
n_layers = 6
dff = d_model * 4
dropout = 0.2
learning_rate = 3e-4
epochs =5_000
eval_iters = 200

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True

In [2]:
data = open('text.txt').read()
vocab = list(sorted(set(data)))
vocab_size = len(vocab)

print(f'Vocab: {vocab[:10]}.')
print(f'Vocab size: {vocab_size}')

stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[i] for i in i])

print(decode(encode(data[1100:1150])))
data = torch.tensor(encode(data))

n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

print(f'Training samples: {train.shape[0]}')
print(f'Validation samples: {val.shape[0]}')

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Xb, Yb = make_batches('train')

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3'].
Vocab size: 65
very dog to the commonalty.

Second Citizen:
Consi
Training samples: 1003854
Validation samples: 111539


In [3]:
class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, d_model, block_size, dropout=0.1):

        super().__init__()
        assert d_model % n_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.n_heads = n_heads
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.key = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)
        self.att_proj = nn.Linear(d_model, d_model, bias=False)
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size), diagonal=1).bool())

    def forward(self, x):

        q = x
        k = x
        v = x
        B,T,_ = x.shape 
        dk = self.d_model // self.n_heads

        # linear projections
        q = self.query(q) 
        k = self.key(k) 
        v = self.value(v) 

        # add number of heads
        q = q.view(B,T,self.n_heads,dk).permute(0,2,1,3)   # B,T,h,dk
        k = k.view(B,T,self.n_heads,dk).permute(0,2,1,3)  
        v = v.view(B,T,self.n_heads,dk).permute(0,2,1,3)  
        
        # attention 

        x = q @ k.transpose(-2,-1) # B,h,T,dk @ B,h,dk,T --> B,h,T,T
        x = x * dk ** -0.5 # B,h,T,T
        x = x.masked_fill(self.mask, float('-inf')) # B,h,T,T
        x = self.dropout(F.softmax(x, dim=(-1)))
        x = x @ v  # B,h,T,T @ B,T,h,dv --> B,h,T,dv
        B,h,T,dv = x.shape
        x = x.transpose(2,1).contiguous().view(B,T,h*dv) #B,T,C
        out = self.dropout(self.att_proj(x)) # B,T,C

        return out

In [4]:
class FeedForward(nn.Module):

    def __init__(self, d_model, dff, dropout, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.dff = nn.Linear(d_model, dff)
        self.out = nn.Linear(dff, d_model)
        self.dropout = nn.Dropout(dropout)

    
    def forward(self, x):

        x = F.gelu(self.dff(x)) # B,T,C (dff)
        x = self.dropout(x) 
        x = self.out(x) # B,T,C (d_model)

        return x

In [5]:
class DecoderBlock(nn.Module):

    def __init__(self, n_heads, d_model, dff, dropout=0.1, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.attention = MultiHeadAttention(n_heads=n_heads, 
                                            d_model=d_model, 
                                            dropout=dropout,
                                            block_size=block_size)
        self.ffl = FeedForward(d_model, dff, dropout)
        self.lnorm = nn.LayerNorm(d_model)
        self.lnorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        att_out = self.attention(x)
        x = self.lnorm(x + self.dropout(att_out))
        ffl_out = self.lnorm2(self.ffl(x))
        x = x + self.dropout(ffl_out)
        return x


In [6]:
class Decoder(nn.Module):

    def __init__(self, n_heads, d_model, dff, n_layers, dropout=0.1, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.decoder = nn.ModuleList([DecoderBlock(n_heads=n_heads,
                                                 d_model=d_model,
                                                 dff=dff,
                                                 dropout=dropout,)
                                                 for l in range(n_layers)])

    def forward(self, x):

        for block in self.decoder:
            x = block(x)

        return x

In [7]:
class Model(nn.Module):

    def __init__(self, d_model, vocab_size, block_size, n_layers, n_heads, dff, dropout = 0.1, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.d_model = d_model
        self.block_size = block_size

        self.embeddings = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(block_size, d_model)
        self.decoder = Decoder(n_heads=n_heads,
                                    d_model=d_model,
                                    dff=dff,
                                    n_layers=n_layers,
                                    dropout=dropout)
        self.lnorm = nn.LayerNorm(vocab_size)
        self.out = nn.Linear(d_model, vocab_size, bias=False)

    def forward(self, x, targets=None):

        B, T = x.shape
        tok_emb = self.embeddings(x) * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
        pos_emb = self.pos_embed(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb
        x = self.decoder(x)
        logits = self.lnorm(self.out(x))

        if targets == None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        B, T = idx.shape
        if T < self.block_size:
            # pad the input with zeros if it's less than block_size
            idx = F.pad(idx, (0, self.block_size - T))
        for _ in range(max_new_tokens):
            # use only the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [8]:
m = Model(vocab_size = vocab_size, 
          block_size = block_size, 
          n_layers = n_layers, 
          n_heads = n_heads,
          dropout = dropout,
          d_model = d_model,
          dff = dff
          ).to(device)


optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
n_params = sum(p.nelement() for p in m.parameters())
print(m)
print(f'Total parameters: {n_params}')

@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = make_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

Model(
  (embeddings): Embedding(65, 512)
  (pos_embed): Embedding(256, 512)
  (decoder): Decoder(
    (decoder): ModuleList(
      (0-5): 6 x DecoderBlock(
        (attention): MultiHeadAttention(
          (dropout): Dropout(p=0.2, inplace=False)
          (query): Linear(in_features=512, out_features=512, bias=False)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=False)
          (att_proj): Linear(in_features=512, out_features=512, bias=False)
        )
        (ffl): FeedForward(
          (dff): Linear(in_features=512, out_features=2048, bias=True)
          (out): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (lnorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (lnorm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (lnor

In [9]:
# Training
for epoch in range(epochs):

    Xb, Yb = make_batches('train')
    logits, loss = m(Xb, Yb) # B, C

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 999:
        l = estimate_loss(m)
        print(f"Iteration {epoch}. Training Loss: {l['train']:.3f}. Evaluation Loss: {l['val']:.3f}")

Iteration 999. Training Loss: 1.875. Evaluation Loss: 2.010
Iteration 1999. Training Loss: 1.531. Evaluation Loss: 1.724
Iteration 2999. Training Loss: 1.399. Evaluation Loss: 1.625
Iteration 3999. Training Loss: 1.330. Evaluation Loss: 1.568
Iteration 4999. Training Loss: 1.280. Evaluation Loss: 1.532


In [10]:
max_new_tokens = 2_000
seed_text = ". "
seed_idx = torch.tensor([stoi[c] for c in seed_text], device=device).unsqueeze(0)
predictions = m.generate(seed_idx, max_new_tokens).to(device)
pad_len = m.block_size
generated_text = decode(predictions[0].tolist())
generated_text = generated_text[pad_len:]  # Remove leading padding
print(generated_text)

SABTYh

ANoT.v
CaNiMANO;CATsThe
MA.HoIClon
TovAnoAldWYounoUwCAQUbadAUjureckire:
ThecoCllllllllloy Mut:
Yora LAwnd:
Pray hecos:
Ay, madaly, y:
I hedd, ha I y iUn have thank a!
An GolloWes yow with teful sen his oul:
A laidiseter.

 SICINIZIUS:'tis thousand the fielded not put apon.

AU, my lord, your, what was you leave artichury,
Carios Margaret
At thousan'd friends,
Break your heread with you.

CLIFFORD:
This sovereigns doth encre time on ans,yer well,
and I dorom ho! Wrwithin fray the cords ways
Supositined to his ame; 'Tush wit nowYork's these
Will we semser'd--'Go too die the co:'
And nell bet. See, e, good this prety.
The eexchance mal I far a pices in the rong
Light ort steeps.

FLORIZEL:
Hence thou rackst me be giled with him:
What and by sigciven do crnowine best;
But turnsling his broygives the basen
The lifeVst that a carening of this firms,
terrfor this armour comm break of est at his remotion inted,
Two depostures: do yourselved to so thKenowing RosparesRumo trum.

KENRCY V